In [ ]:
# this script converts image segmentation masks, aka frame by frame "cut-outs" of a hitter's body and bat, into sets of coordinates that can be used to plot the outline
# once I have enough frame by frame coordinates, I will use those to train an image/video segmentation model so I no longer need to manually annotate players' bodies
# eventually where this leads is the employment of the methods in this paper: "Human Motion Recognition Using Isomap and Dynamic Time Warping" from 2007

# Copyright (C) 2025 Ryan Gunther
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.
#
# You should have received a copy of the GNU Affero General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.
#
# Modified from: https://github.com/computervisioneng/image-segmentation-yolov8
# Modifications by Ryan Gunther, 2025

In [ ]:
from google.colab import files
uploaded = files.upload()  # pick files or a zip

Saving task_1578568_annotations_2025_08_30_02_26_36_segmentation mask 1.1.zip to task_1578568_annotations_2025_08_30_02_26_36_segmentation mask 1.1.zip


In [ ]:
import zipfile, io
z = zipfile.ZipFile(io.BytesIO(next(iter(uploaded.values()))))
z.extractall("/content/data")

In [ ]:
import os

import cv2


input_dir = '/content/data/SegmentationObject'
output_dir = '/content/labels'
os.makedirs(output_dir, exist_ok=True)

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    # load the binary mask and get its contours
    mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convert the contours to polygons
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)
            polygons.append(polygon)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
        for polygon in polygons:
            for p_, p in enumerate(polygon):
                if p_ == len(polygon) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()